In [1]:
import sys
import os
import random
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"
if os.path.exists('./core'):
    os.remove('./core')

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
    sys.path.append('../symbolicregression/')

# Install

In [ ]:
%%capture
if IN_COLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    !git clone https://github.com/facebookresearch/symbolicregression symbolic
    %mv ./symbolic/* ./
    %rm -rf symbolic
    !conda env create --name symbolic regression --file=environment.yml
    !conda init
    !activate symbolic
    !pip install git+https://github.com/pakamienny/sympytorch
    !conda install -c conda-forge pysr
    !conda install -c conda-forge julia
    !python3 -m pysr install

In [ ]:
import pysr
from pysr import PySRRegressor
pysr.julia_helpers.init_julia()

### Transformer Basline

In [ ]:
import torch
import pandas as pd
import numpy as np
import sympy as sp
import symbolicregression
import requests
from matplotlib import pyplot as plt

params = {'text.usetex' : False,
          'font.size' : 36,
          'legend.fancybox':True,
          'legend.loc' : 'best',

          'legend.framealpha': 0.9,
          "legend.fontsize" : 21,
         }
plt.rcParams.update(params)
from sympy.printing import latex
from IPython.display import display
from time import perf_counter
from math import pi, cos, sin , sqrt , exp , atan , atan2

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from sympy import Number
def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(Number)})

In [ ]:
def set_seed(seed=4242):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
set_seed()

# Known Potential experiment

In [ ]:
def get_latin(scale=5, parameter_static_x = 2.8, parameter_static_y = 2.5):
    import random
    
    from scipy.stats.qmc import LatinHypercube
    engine = LatinHypercube(d=2)
    samples = engine.random(n_samples)*scale
    x_coords = samples[:, 0]
    y_coords = samples[:, 1]
    arr = []
    
    for x,y in zip(x_coords, y_coords):
        distance = (parameter_static_x-x)**2/ 0.5**2 + (parameter_static_y-y)**2 / 0.5**2
        #distance = (paramters_static[0]-simX[i,0])**2/(0.2+radius)**2 + (paramters_static[0]-simX[i,1])**2 / (0.2+radius)**2
        #distance = (parameter_static_x-x)**2/(0.2+radius)**2 + (parameter_static_y-y)**2 / (0.2+radius)**2

        obst_stat = 5*(pi/2 + atan(w2 - distance*w2))
        arr.append([obst_stat, x, y])
    robot_df = pd.DataFrame(arr)
    return robot_df

In [ ]:
def get_grid(scale=5,n_samples=300, 
             parameter_static_x = 2.5, 
             parameter_static_y = 2.5,
             th = 0.5,
             w2=10):
    arr = []
    obs_list = []
    n_samples=int(n_samples**0.5)
    # Создаем поле размером 5x5
    field_x = np.linspace(0, scale, num=n_samples) # change this if need other numbers of points
    field_y = np.linspace(0, scale, num=n_samples)
    # Получаем координаты всех точек в поле
    points_x, points_y = np.meshgrid(field_x, field_y)
    scaled_points_x = points_x.flatten()
    scaled_points_y = points_y.flatten()
    # Создаем массив уникальных точек
    points = list(zip(scaled_points_x , scaled_points_y))
    for x,y in zip(scaled_points_x, scaled_points_y):

        distance = (parameter_static_x-x)**2/ 0.5**2 + (parameter_static_y-y)**2 / 0.5**2
        obst_stat = 5*(pi/2 + atan(w2 - distance*w2))
        arr.append([obst_stat, x, y])
        obs_list.append(obst_stat)

    points = list(zip(scaled_points_x , scaled_points_y))
    arr = np.array(arr)
    indeces = np.where(arr[:,0] < th)[0]
    indeces = np.random.choice(indeces, round(len(indeces)*0.005))
    indeces2 = np.where(arr[:,0] >= th)[0]
    arr = np.concatenate ((arr[indeces], arr[indeces2]))
    robot_df = pd.DataFrame(arr)
    return robot_df

In [ ]:
def get_robot_data(from_latin=True,from_file=False,
                   from_grid=False,
                   from_sample=False,
                   n_samples=300,
                   radius=0.3,
                   w2=10,
                   file_path='./data/hyper_random_xy.csv',
                   step=2,parameter_static_x=2.5,parameter_static_y=2.5):
    if from_file:
        robot_df=pd.read_csv(file_path,sep=',',header=None).dropna(axis=0).astype('float32')
        robot_df.columns='j2,x,y'.split(',')
        robot_df = robot_df[robot_df['j2'] > 1]
    elif from_sample:
        robot_df = robot_data_sample['j2,x,y'.split(',')]
    elif from_grid:
        robot_df = get_grid(n_samples=n_samples,parameter_static_x = parameter_static_x, 
             parameter_static_y = parameter_static_y)
    else:
        robot_df = get_latin(n_samples=n_samples)
    robot_df.columns='j2,x,y'.split(',')
    robot_df = robot_df.sort_values('j2')[::]
    return robot_df

In [ ]:
n_samples=20000-1
df = get_robot_data(from_sample=False,from_grid=True,
                    n_samples=n_samples,
                    parameter_static_x=3,parameter_static_y=2).astype('float32')
df = df.reset_index().drop('index',1)
n_train = int(len(df)*0.9)
train_index = np.random.choice(df.index,n_train)
test_index = np.array([i for i in df.index if i not in train_index])
robot_df_train = df.iloc[train_index]
robot_df_test = df.iloc[test_index]


In [ ]:
model_path = "/tmp/model.pt"
try:
    if not os.path.isfile(model_path):
        url = "https://dl.fbaipublicfiles.com/symbolicregression/model1.pt"
        r = requests.get(url, allow_redirects=True)
        open(model_path, 'wb').write(r.content)
    if not torch.cuda.is_available():
        model = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        model = torch.load(model_path)
        model = model.cuda()
    print(model.device)
    print("Model successfully loaded!")

except Exception as e:
    print("ERROR: model not loaded! path was: {}".format(model_path))
    print(e)

transformer = symbolicregression.model.SymbolicTransformerRegressor(
                        model=model,
                        max_input_points=20000,
                        n_trees_to_refine=500,
                        rescale=True
                        )

In [ ]:
P = robot_df_train['x,y'.split(',')].values
j2 = robot_df_train['j2'].values

In [ ]:
t = perf_counter()
transformer.fit(P,j2)
t = perf_counter() - t

In [ ]:
replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/",'x_0':'x','x_1':'y'}
transformer_str = transformer.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
for op,replace_op in replace_ops.items():
    transformer_str = transformer_str.replace(op,replace_op)

In [ ]:
robot_model_transformer = round_expr(sp.parse_expr(transformer_str),5)

In [ ]:
j2_approx_tranfromer=sp.lambdify(['x','y'],robot_model_transformer,"numpy")

In [ ]:
str(robot_model_transformer)

In [ ]:
try:
    approx_train = j2_approx_tranfromer(robot_df_train['x'].values,robot_df_train['y'].values)
    train_err = mse(approx_train,robot_df_train['j2'])
    approx_test = j2_approx_tranfromer(robot_df_test['x'].values,robot_df_test['y'].values)
    test_err = mse(approx_test,robot_df_test['j2'])
    print(train_err,test_err)
except Exception as e:
    print(e)

In [ ]:
fig = plt.figure(figsize=(20,20))
arr = robot_df_train.values
ax = fig.add_subplot(111, projection='3d')
ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_tranfromer(arr[:,1],arr[:,2]),color='red',label='transformer')
ax.scatter(arr[:,1], arr[:,2], 
           arr[:,0],color='green',label='j2')

arr = robot_df_test.values

ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_tranfromer(arr[:,1],arr[:,2]),color='red',label='transformer (test)',marker='x')
ax.scatter(arr[:,1], arr[:,2], 
           arr[:,0],color='green',label='j2',marker='x')

plt.legend()
plt.tight_layout()
plt.show()

#### PySR baseline

In [ ]:

robot_model = PySRRegressor(
    niterations=40,  # < Increase me for better results
    binary_operators=["+",'-', "*",],
    temp_equation_file=True,
    delete_tempfiles=True,
    unary_operators=[
        "atan",
        "exp",
        "sin"
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    # ^ Define operator for SymPy as well
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
)

In [ ]:
t2 = perf_counter()
robot_model.fit(P[:],j2[:])
t2 = perf_counter()-t2

In [ ]:
print(t2)

In [ ]:
if os.path.exists('./core'):
    os.remove('./core')

In [ ]:
save_eq = False

In [ ]:
best_eq_id = robot_model.equations_['loss'].argmin()

In [ ]:
if save_eq:
    robot_model.equations_.to_csv('./'+'robot_model.equations_.csv'.replace('.','_'))

In [ ]:
robot_model_pysr = robot_model.equations_.iloc[best_eq_id]['equation']
robot_model_pysr = str(round_expr(sp.parse_expr(robot_model_pysr),5)).replace('x0','x').replace('x1','y')

In [ ]:
str(robot_model_pysr)

In [ ]:
j2_approx_pysr=sp.lambdify(['x','y'],robot_model_pysr, modules=["numpy"])

In [ ]:
try:
    approx_train = j2_approx_pysr(robot_df_train['x'].values,robot_df_train['y'].values)
    train_err = mse(approx_train,robot_df_train['j2'])
    approx_test = j2_approx_pysr(robot_df_test['x'].values,robot_df_test['y'].values)
    test_err = mse(approx_test,robot_df_test['j2'])
    print(train_err,test_err)
except Exception as e:
    print(e)

In [ ]:
params = {'text.usetex' : False,
          'font.size' : 36,
          'legend.fancybox':True,
          'legend.loc' : 'best',

          'legend.framealpha': 0.9,
          "legend.fontsize" : 21,
         }
plt.rcParams.update(params)

In [ ]:
fig = plt.figure(figsize=(15,15))
arr = robot_df_train.values
s=90
ax = fig.add_subplot(111, projection='3d')
plt.xlabel("x")
plt.ylabel("y")
ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_tranfromer(arr[:,1],arr[:,2]),color='red',label='transformer',s=s)
ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_pysr(arr[:,1],arr[:,2]),color='magenta',label='pysr',s=s)
ax.scatter(arr[:,1], arr[:,2], 
           arr[:,0],color='green',label='j2',s=s)

arr = robot_df_test.values

ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_tranfromer(arr[:,1],arr[:,2]),color='red',marker='x',s=s)
ax.scatter(arr[:,1], arr[:,2], 
           j2_approx_pysr(arr[:,1],arr[:,2]),color='magenta',marker='x',s=s)
ax.scatter(arr[:,1], arr[:,2], 
           arr[:,0],color='green',marker='x',s=s)
ax.scatter(0,0,color='black',marker='o',label='train',s=90)
ax.scatter(0,0,color='black',marker='x',label='test',s=90)

plt.legend()
plt.tight_layout()

In [ ]:
## equations

In [ ]:
print(robot_model_transformer)


In [ ]:
print(robot_model_pysr)


# Costmap Experment

In [ ]:
#code by Muhammad 
import numpy as np
from math import sin , cos , atan2, pi , atan
from matplotlib import colors , transforms
import matplotlib.pyplot as plt


In [ ]:
num_cells = 256
def get_potential(x):
    potential_at_x = (pi/2 + atan(10 - (x-1) * 10)) * 15
    return potential_at_x

def create_cost_map():

    oc_grid = np.zeros((num_cells,num_cells))

    for i in range(20,60):
        for j in range(90,130):
            oc_grid[i,j] = 100
    for i in range(100,180):
        for j in range(175,255):
            oc_grid[i,j] = 100
    for i in range(130,200):
        for j in range(30,100):
            oc_grid[i,j] = 100

    data_obst1 = np.zeros((num_cells,num_cells))

    first_obst_origin = np.array([20,90])
    first_obst_height = 40
    first_obst_width = 40
    data_obst1[first_obst_origin[0],first_obst_origin[1]] = -10

    for k in range (21):
        for i in range (first_obst_origin[0] + k,first_obst_origin[0]+first_obst_height +1 - k):
            data_obst1[i,first_obst_origin[1]+k] = -k
            data_obst1[i,first_obst_origin[1]+first_obst_width-k] = -k

        for i in range (first_obst_origin[1] + k,first_obst_origin[1]+first_obst_width - k):
            data_obst1[first_obst_origin[0]+k,i] = -k
            data_obst1[first_obst_origin[0]+first_obst_height -k, i] = -k

    for k in range (num_cells):
        for i in range (first_obst_origin[0] - k,first_obst_origin[0]+first_obst_height +1 + k):
            if(i<num_cells and i >=0):
                if((first_obst_origin[1]-k)>=0):
                    data_obst1[i,first_obst_origin[1]-k] = k
                if((first_obst_origin[1]+first_obst_width+k)<num_cells):
                    data_obst1[i,first_obst_origin[1]+first_obst_width+k] = k

        for i in range (first_obst_origin[1] - k,first_obst_origin[1]+first_obst_width + k):
            if(i<num_cells and i >=0):
                if((first_obst_origin[0]-k)>=0):
                    data_obst1[first_obst_origin[0]-k,i] = k
                if((first_obst_origin[0]+first_obst_height +k)<num_cells):
                    data_obst1[first_obst_origin[0]+first_obst_height +k, i] = k

    data_obst2 = np.zeros((num_cells,num_cells))


    second_obst_origin = np.array([100,175])
    second_obst_height = 80
    second_obst_width = 80
    data_obst2[second_obst_origin[0],second_obst_origin[1]] = -10

    for k in range (21):
        for i in range (second_obst_origin[0] + k,second_obst_origin[0]+second_obst_height +1 - k):
            data_obst2[i,second_obst_origin[1]+k] = -k
            data_obst2[i,second_obst_origin[1]+second_obst_width-k] = -k

        for i in range (second_obst_origin[1] + k,second_obst_origin[1]+second_obst_width - k):
            data_obst2[second_obst_origin[0]+k,i] = -k
            data_obst2[second_obst_origin[0]+second_obst_height -k, i] = -k

    for k in range (num_cells):
        for i in range (second_obst_origin[0] - k,second_obst_origin[0]+second_obst_height +1 + k):
            if(i<num_cells and i >=0):
                if((second_obst_origin[1]-k)>=0):
                    data_obst2[i,second_obst_origin[1]-k] = k
                if((second_obst_origin[1]+second_obst_width+k)<num_cells):
                    data_obst2[i,second_obst_origin[1]+second_obst_width+k] = k

        for i in range (second_obst_origin[1] - k,second_obst_origin[1]+second_obst_width + k):
            if(i<num_cells and i >=0):
                if((second_obst_origin[0]-k)>=0):
                    data_obst2[second_obst_origin[0]-k,i] = k
                if((second_obst_origin[0]+second_obst_height +k)<num_cells):
                    data_obst2[second_obst_origin[0]+second_obst_height +k, i] = k


    data_obst3 = np.zeros((num_cells,num_cells))

    third_obst_origin = np.array([130,30])
    third_obst_height = 70
    third_obst_width = 70
    data_obst3[third_obst_origin[0],third_obst_origin[1]] = -10

    for k in range (21):
        for i in range (third_obst_origin[0] + k,third_obst_origin[0]+third_obst_height +1 - k):
            data_obst3[i,third_obst_origin[1]+k] = -k
            data_obst3[i,third_obst_origin[1]+third_obst_width-k] = -k

        for i in range (third_obst_origin[1] + k,third_obst_origin[1]+third_obst_width - k):
            data_obst3[third_obst_origin[0]+k,i] = -k
            data_obst3[third_obst_origin[0]+third_obst_height -k, i] = -k

    for k in range (num_cells):
        for i in range (third_obst_origin[0] - k,third_obst_origin[0]+third_obst_height +1 + k):
            if(i<num_cells and i >=0):
                if((third_obst_origin[1]-k)>=0):
                    data_obst3[i,third_obst_origin[1]-k] = k
                if((third_obst_origin[1]+third_obst_width+k)<num_cells):
                    data_obst3[i,third_obst_origin[1]+third_obst_width+k] = k

        for i in range (third_obst_origin[1] - k,third_obst_origin[1]+third_obst_width + k):
            if(i<num_cells and i >=0):
                if((third_obst_origin[0]-k)>=0):
                    data_obst3[third_obst_origin[0]-k,i] = k
                if((third_obst_origin[0]+third_obst_height +k)<num_cells):
                    data_obst3[third_obst_origin[0]+third_obst_height +k, i] = k


    cost_map = np.zeros((num_cells,num_cells))

    for i in range(num_cells):
        for j in range(num_cells):
            cost_map[i,j] = 0.02 * min(data_obst1[i,j],data_obst2[i,j],data_obst3[i,j])

    cost_map = cost_map + 1


    cmap = colors.ListedColormap(['white' , 'black', 'red','green' ])
    fig, ax = plt.subplots(figsize=(5,5))
    ax.pcolor(oc_grid[::-1],cmap=cmap,edgecolors='w', linewidths=0.1)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    plt.show()

    return cost_map


In [ ]:
num_cells = 256
def get_potential(x):
    potential_at_x = (pi/2 + atan(10 - (x-1) * 10)) * 15
    return potential_at_x

def create_cost_map():

    oc_grid = np.zeros((num_cells,num_cells))

    for i in range(20,60):
        for j in range(90,130):
            oc_grid[i,j] = 100
    for i in range(100,180):
        for j in range(175,255):
            oc_grid[i,j] = 100
    for i in range(130,200):
        for j in range(30,100):
            oc_grid[i,j] = 100

    data_obst1 = np.zeros((num_cells,num_cells))

    first_obst_origin = np.array([20,90])
    first_obst_height = 40
    first_obst_width = 40
    data_obst1[first_obst_origin[0],first_obst_origin[1]] = -10

    for k in range (21):
        for i in range (first_obst_origin[0] + k,first_obst_origin[0]+first_obst_height +1 - k):
            data_obst1[i,first_obst_origin[1]+k] = -k
            data_obst1[i,first_obst_origin[1]+first_obst_width-k] = -k

        for i in range (first_obst_origin[1] + k,first_obst_origin[1]+first_obst_width - k):
            data_obst1[first_obst_origin[0]+k,i] = -k
            data_obst1[first_obst_origin[0]+first_obst_height -k, i] = -k

    for k in range (num_cells):
        for i in range (first_obst_origin[0] - k,first_obst_origin[0]+first_obst_height +1 + k):
            if(i<num_cells and i >=0):
                if((first_obst_origin[1]-k)>=0):
                    data_obst1[i,first_obst_origin[1]-k] = k
                if((first_obst_origin[1]+first_obst_width+k)<num_cells):
                    data_obst1[i,first_obst_origin[1]+first_obst_width+k] = k

        for i in range (first_obst_origin[1] - k,first_obst_origin[1]+first_obst_width + k):
            if(i<num_cells and i >=0):
                if((first_obst_origin[0]-k)>=0):
                    data_obst1[first_obst_origin[0]-k,i] = k
                if((first_obst_origin[0]+first_obst_height +k)<num_cells):
                    data_obst1[first_obst_origin[0]+first_obst_height +k, i] = k

    data_obst2 = np.zeros((num_cells,num_cells))


    second_obst_origin = np.array([100,175])
    second_obst_height = 80
    second_obst_width = 80
    data_obst2[second_obst_origin[0],second_obst_origin[1]] = -10

    for k in range (21):
        for i in range (second_obst_origin[0] + k,second_obst_origin[0]+second_obst_height +1 - k):
            data_obst2[i,second_obst_origin[1]+k] = -k
            data_obst2[i,second_obst_origin[1]+second_obst_width-k] = -k

        for i in range (second_obst_origin[1] + k,second_obst_origin[1]+second_obst_width - k):
            data_obst2[second_obst_origin[0]+k,i] = -k
            data_obst2[second_obst_origin[0]+second_obst_height -k, i] = -k

    for k in range (num_cells):
        for i in range (second_obst_origin[0] - k,second_obst_origin[0]+second_obst_height +1 + k):
            if(i<num_cells and i >=0):
                if((second_obst_origin[1]-k)>=0):
                    data_obst2[i,second_obst_origin[1]-k] = k
                if((second_obst_origin[1]+second_obst_width+k)<num_cells):
                    data_obst2[i,second_obst_origin[1]+second_obst_width+k] = k

        for i in range (second_obst_origin[1] - k,second_obst_origin[1]+second_obst_width + k):
            if(i<num_cells and i >=0):
                if((second_obst_origin[0]-k)>=0):
                    data_obst2[second_obst_origin[0]-k,i] = k
                if((second_obst_origin[0]+second_obst_height +k)<num_cells):
                    data_obst2[second_obst_origin[0]+second_obst_height +k, i] = k


    data_obst3 = np.zeros((num_cells,num_cells))

    third_obst_origin = np.array([130,30])
    third_obst_height = 70
    third_obst_width = 70
    data_obst3[third_obst_origin[0],third_obst_origin[1]] = -10

    for k in range (21):
        for i in range (third_obst_origin[0] + k,third_obst_origin[0]+third_obst_height +1 - k):
            data_obst3[i,third_obst_origin[1]+k] = -k
            data_obst3[i,third_obst_origin[1]+third_obst_width-k] = -k

        for i in range (third_obst_origin[1] + k,third_obst_origin[1]+third_obst_width - k):
            data_obst3[third_obst_origin[0]+k,i] = -k
            data_obst3[third_obst_origin[0]+third_obst_height -k, i] = -k

    for k in range (num_cells):
        for i in range (third_obst_origin[0] - k,third_obst_origin[0]+third_obst_height +1 + k):
            if(i<num_cells and i >=0):
                if((third_obst_origin[1]-k)>=0):
                    data_obst3[i,third_obst_origin[1]-k] = k
                if((third_obst_origin[1]+third_obst_width+k)<num_cells):
                    data_obst3[i,third_obst_origin[1]+third_obst_width+k] = k

        for i in range (third_obst_origin[1] - k,third_obst_origin[1]+third_obst_width + k):
            if(i<num_cells and i >=0):
                if((third_obst_origin[0]-k)>=0):
                    data_obst3[third_obst_origin[0]-k,i] = k
                if((third_obst_origin[0]+third_obst_height +k)<num_cells):
                    data_obst3[third_obst_origin[0]+third_obst_height +k, i] = k


    cost_map = np.zeros((num_cells,num_cells))

    for i in range(num_cells):
        for j in range(num_cells):
            cost_map[i,j] = 0.02 * min(data_obst1[i,j],data_obst2[i,j],data_obst3[i,j])

    cost_map = cost_map + 1


    cmap = colors.ListedColormap(['white' , 'black', 'red','green' ])
    fig, ax = plt.subplots(figsize=(5,5))
    ax.pcolor(oc_grid[::-1],cmap=cmap,edgecolors='w', linewidths=0.1)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    plt.show()

    return cost_map


In [ ]:
cost_map  = create_cost_map()
plotential_matrix= np.zeros((num_cells,num_cells))
for i in range(num_cells):
    for j in range(num_cells):
        plotential_matrix[i,j] = get_potential(cost_map[i,j])

X = np.linspace(0, 5.12, num_cells)
Y = np.linspace(0, 5.12, num_cells)
X, Y = np.meshgrid(X, Y)


# # show image potential of cost map
plotting_plotential_matrix = np.zeros((num_cells,num_cells))
for i in range(num_cells):
    for j in range(num_cells):
        plotting_plotential_matrix[num_cells-1-i,j] = plotential_matrix[i,j]
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X, Y, plotting_plotential_matrix, 50, cmap='binary')
plt.show()

In [ ]:
plotting_plotential_matrix.shape